# Линейная алгебра. Вектор (10 баллов)

В этой домашней работе мы попробуем разобраться в том, что же такое векторы, матрицы и тензоры, как они работают и что можно делать с их помощью. Обычно для этих целей используются библиотеки `numpy`, `tensorflow` и `pytorch`, на основании которых написано еще много чего интересного. Можно было бы воспользоваться ими и пробовать вызывать разные методы, пытаясь понять, как они работают. Но это не наш путь, поэтому мы напишем все сами.

Писать мы будет простенький класс `Tensor`, который представляет собой типизированный многомерный массив. В этой работе мы рассмотрим только случаи с тензорами размерности 1, то есть с обычными векторами.

`Tensor` представляет собой простую обертку над стандартным типизированным питоновским массивом. Внутри он хранит значения элементов переданного ему в конструктор списка, для этого используется стандартный питоновский контейнер `array`. Тип хранимых значений -- `float32`, это числа с плавающей точкой, для хранения которых используется четыре байта (32 бита).

В процессе работы мы будем подворовывать интерфейс библиотеки `pytorch`. Если торч это большой факел, то наша библиотека это маленький огонек по размерам, поэтому назовем ее `twinkle` :)

## Что уже сделано

Частично функционал класса `Tensor` уже реализован, а именно та его часть, которая отвечает за хранение данных. Сделано это в файле `twinkle/tensor.py`. Что уже есть в авторском коде (не поленитесь изучить его, там немного):

1. Конструктор, принимающий число, реализующее интерфейс `number.Number`. Он создает тензор нулевой размерности, хранящий это число в поле `_data`;
2. Конструктор, принимающий объект, реализующий интерфейс `Iterable`. Этот итератор должнен обходить только числа с интерфейсом `numbers.Number`;
3. Магический метод `__repr__`, который умеет красиво содержимое тензора. Если хочется, его вывод можно скопировать и вставить в код, получив копию объекта;
4. Метод `allclose`, который сравнивает два тензора по значению;
5. Функция `randn`, которая возвращает объект `Tensor` заданной формы, заполняя его координаты случайными значениями из нормального распределения $\mathcal{N}(0, 1)$;

## Что будем делать

Мы умеем хранить векторы, но нам не хватает операций над ними. Наша маленькая библиотека должна уметь складывать и вычитать векторы, домножать их на скаляры и уметь вычислять скалярное произведение. Если тезисно, то реализовать нужно:

7. Свойство `shape`, которое возвращает кортеж с размеростями тензора. В нашем случае для вектора это кортеж из одного числа -- длины вектора;
8. Функцию `vector_norm`, которая вычисляет метрику Минковского для векторов;
9. Методы `add`, `mul` и `sub` реализующие сложение тензора с числом/нульмерным тензором/другим тензором той же размерности;
10. Методы `gt` *(greater than)* и `lt` *(less than)*, по-русски больше или меньше, которые реализуют покоординатное сравнение с числом/нульмерным тензором/другим тензором;
11. Метод `neg`, меняющий знак у каждого элемента тензора;
12. Магические методы `__add__`, `__mul__`, `__sub__`, перегружающие операторы `+`, `*`, `-` и действующие так же, как и методы `add`, `mul`, `sub`, например `tensor + other`, `tensor * other` и `tensor - other`;
13. Магические методы `__radd__`, `__rmul__`, `__rsub__`, которые отличаются правоассоциативностью, в отличие от пункта выше. Выглядит они так же, но тензор стоит после оператора, например `other + tensor`, `other * tensor` и `other - tensor`.
14. Магические методы `__gt__` и `__lt__`, перегружающие операторы `>` и `<` и действующие так же, как и методы `gt` и `lt`, например `tensor > other` или `tensor < other`;
15. Магический метод `__neg__`, перегружающий унарный `-` и и действующий как метод `neg`;
16. Магический метод `__len__`, возвращающий количество элементов в тензоре по первой размерности;
17. Магический метод `__eq__`, перегружающий оператор `==` и вызывающий `allclose`;
18. Магический метод `__iter__`, который реализует итератор по элементам тензора учитывая только первую размерность;
19. Магический метод `__getitem__`, который возвращает `i-ю` координату вектора;
20. Функции `add`, `mul` и `sub`, повторящие функционал соответствующих методов тензора;
21. Функции `zeros`, `ones` и `tensor`, создающие тензор из нулей/единиц/списка чисел.

Для всего этого написаны тесты, которые помогут вам проверить правильность вашего кода. Что ж, теперь начнем по порядку :)

# Обвязка класса, вспомогательные метода (2 балла)

Будем честны: сейчас наш класс `Tensor` *не умеет ничего*. Хотелось бы, чтобы можно было не только создавать тензоры, но и хотя бы что-то с ними делать, для начала -- узнать, что же мы создали! Для этого нужно дописать реализацию нескольких крайне полезных вещей:

* 1. Свойство `shape` для того, чтобы узнать форму тензора;
* 2. Магический метод `__len__` для того, чтобы узнать количество элементов в тензоре;
* 3. Магический метод `__getitem__` или индексер для получения *i-го* элемент тензора-вектора через оператор `[]`;
* 4. Магический метод `__iter__` или итератор для обхода элементов тензора по порядку в цикле `for .. in`

## Свойство `shape` (0.5 балла)

Начнем с простого. Сейчас форма вектора лежит в кортеже, внутри свойства `_shape` класса `Tensor`. Это свойство приватное, и *нельзя* изменять его вне методов класса. Реализуйте доступное только для чтения *свойство* `shape` объекта класса `Tensor`, которое будет возвращать шейп тензора. Оно выглядит как метод класса, только на строчку выше написано `@property`. 

После этого, чтобы удостоверится в правильности реализации кода, прогоните тесты для проверки. Это можно сделать как в ячейке ниже, так и в консоли, запустив следующую команду:

`pytest . -k test_shape -v`

Для запуска в ячейке перед командой введите восклицательный знак (просто запустите ячейку ниже, он там уже есть). Для проверки конечно было бы удобно, если бы все такие ячейки были выполнены.

In [ ]:
!pytest . -k test_shape -v

Как понять, что все прошло успешно? Есть несколько признаков:

* `PASSED` -- справа от всех тестов написано `PASSED`;
* `[100%]` -- еще правее цифра 100%;   
* В полосе снизу написано `1 passed`, нет слова `failed` и она зеленого цвета, а не красного. 

Если что-то пошло не так, проверьте свой код и посмотрите в тесты еще раз. Повторяйте до сходимости, после чего переходите к следующему пункту.

## Магический метод `__len__`, функция `len` (0.5 балла)

Вы можете удивиться, увидев эти подчеркивания и спросить, зачем они? Методы с двумя подчеркиваниями в названии называются магическими *(magic methods)* и служат либо для перегрузки операторов, либо для расширения функциональности класса на средства языка. Напрямую их как правило никто не вызывает, а вызываются они в нужном контексте средствами языка. В данном случае метод `__len__` позволяет починить фукнцию `len` так, чтобы она возвращала количество элементов в тензоре. Реализуйте эту функцию, возвращая из нее количество элементов в тензоре.

*Вопрос: это можно сделать двумя путями с одинаковым результатом. Какой вариант вам нравится больше и почему?*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_len -v`

In [ ]:
!pytest . -k test_len -v

## Магический метод `__getitem__`, индексер (0.5 балла)

Важной особенностью питоновских списков является возможность произвольного доступа к элементу списка, используя оператор `[]`. Для определения такого поведения на объекте произвольного класса у этого класса нужно определить метод `__getitem__`. Метод принимает два аргумента: первый -- это `self`, текущий тензор, до его координат можно добраться через `self.__data`. Второй аргумент это `key` -- целое число, индекс нужного элемента (внутри функции есть отдельная проверка на это, пишите код внутри соответствующего условия). Эта функция должна возвращать *i-й* элемент вектора, где *i* это `key`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_getitem -v`

In [ ]:
!pytest . -k test_getitem -v

## Магический метод `__iter__`, итератор (0.5 балла)

Еще одним важной особенностью языка является возможность обхода списков в цикле `for .. in`. В нашем случае в цикле было бы удобно перебирать координаты вектора. Для реализации такой возможности класс `Tensor` отнаследован от `collections.abc.Iterable` и у него определен метод `__iter__`. Этот метод представляет из себя *генератор*, и возвращает по одному элементу за раз. Перебирайте все элементы внутреннего буфера и возвращайте их через оператор `yield`, а не `return` как в обычных функциях. Делать это нужно на каждой итерации цикла в процессе обхода.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_iter -v`

In [ ]:
!pytest . -k test_iter -v

## Примеры использования

Воспользуемся только что написанными кодом и посмотрим, как *уже* можно пользоваться нашим тензором.

In [ ]:
from twinkle import Tensor

a = Tensor([0, 1, 2, 3])
b = Tensor([4, 5, 6])

print('a.shape', a.shape)
print('b.shape', b.shape, '\n')

print('a[1]', a[1])
print('a[3]', a[3])
print('b[0]', b[0])
print('b[2]', b[2], '\n')

for x in b:
    print(x)

# Векторные нормы (2 балла)

Следующий этап -- вычисление векторных норм (или длин, если вам так проще). Для этого реализуйте функцию `vector_norm`, который будет вычислять $l_1$ и $l_2$ нормы вектора. Заготовку для нее можно найти в файле `twinkle/linalg.py`. Эта функция принимает единственный аргумент `ord`, который может принимать значения 1 или 2 в зависимости от типа нормы (1 для манхэттенского расстояния и 2 для евклидовой нормы).

## $l_2$ норма, метрика $L_2$, евклидова норма, `ord=2` (0.5 балла)

Как много слов в заголовке, да? На самом деле, это все синонимы длины вектора в школьном понимании. Эта норма является геометрическим расстоянием между двумя точками в многомерном пространстве, вычисляемым по теореме Пифагора. Для вычисления этой нормы параметр `ord` должен быть равен двойке. Внутри метода просто вычислите норму по формуле:

$$\left\|\mathbf{x}\right\|_2 = \sqrt{\sum_{i=0}^{n}x_i^2}$$

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_vector_norm_l2 -v`

In [ ]:
!pytest . -k test_vector_norm_l2 -v

## $l_1$ норма, метрика $L_1$, манхэттенское расстояние, `ord=1` (0.5 балла)

Слов в заголовке еще больше, но это не страшно. Для вектора эта норма представляет собой сумму модулей всех его элементов. Для вычисления этой нормы параметр `ord` должен быть равен двойке. Внутри метода просто вычислите норму по формуле:

$$\left\|\mathbf{x}\right\|_1 = \sum_{i=0}^{n}|x_i|$$

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_norm_l1 -v`

In [ ]:
!pytest . -k test_vector_norm_l1 -v

## Бонус: $p$-норма, метрика Минковского, `ord=p` (0.5 балла)

Классическим решением предыдущих двух пунктов является написание условий внутри `vector_norm` в зависимости от переданного `ord`. Более внимательные люди могут заметить, что это две нормы объединяет единая формула $p$-нормы:

$$\left\|\mathbf{x}\right\|_p = \left( \sum_{i=1}^n \left|x_i\right|^p\right)^{1/p}$$

На картинке ниже изображены шары при различных значениях p: расстояния от центра координат до любой точки на синей линии одинаковы при постоянном значении p.

![p-norm](./2D_unit_balls.png)

Переделайте свое решение с учетом того, что в этой формуле `ord` это $p$. Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_p_norm -v`

In [ ]:
!pytest . -k test_p_norm -v

## Магический метод `__eq__`, перегрузка оператора `==` (0.5 балла)

После появления функции для вычисления векторных норм должен заработать метод `allclose`, который поэлементно сравнивает два вектора. Внутри он использует функцию `vector_norm` и принимает два аргумента -- абсолютный и относительный пороги для вычисления равенства двух чисел с плавающей точкой. Точную формулу можно подглядеть либо в коде, либо в документации к [torch.allclose](https://pytorch.org/docs/stable/generated/torch.allclose.html). Вам же нужно вызвать его в магическом методе `__eq__`, просто передав внутрь аргумент `other`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_eq -v`

In [ ]:
!pytest . -k test_eq -v

## Примеры использования

Попробуем создать несколько векторов, вычислить их нормы и сравнить их покоординатно:

In [ ]:
from twinkle import Tensor
from twinkle.linalg import vector_norm

foo = Tensor(range(6))
print(foo)
print(f'l1 norm {vector_norm(foo, ord=1)}')
print(f'l2 norm {vector_norm(foo, ord=2)}')
print(f'lp norm {vector_norm(foo, ord=1.41)}')

# Функции-конструкторы (1.5 баллов)

Чтобы немножко абстрагировать создание тензора и облегчить интерфейс предлагается написать несколько конструкторов. Часто нужно создать тензор нулей или единиц, а также тензор из заранее заданного списка. Для начала реализуем для этого три разные функции, `zeros`, `ones` и `tensor`.

## Конструктор `zeros` (0.5 балла)

Реализуйте функцию-конструктор `zeros` в файле `twinkle/ops.py`, которая создает заполненный нулями тензор заданной размерности (в нашем случае это нулевой вектор).  

*Подсказка: создайте список из `shape[0]` нулей и передайте его в конструктор `Tensor`.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_zeros -v`

In [ ]:
!pytest . -k test_zeros -v

## Конструктор `ones` (0.5 балла)

Реализуйте функцию-конструктор `ones` в файле `twinkle/ops.py`, которая создает заполненный единицами тензор заданной размерности (в нашем случае это единичный вектор).  

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_ones -v`

In [ ]:
!pytest . -k test_ones -v

## Конструктор `tensor` (0.5 балла)

Реализуйте функцию-конструктор `tensor` в файле `twinkle/ops.py`, которая принимает итерируемый список чисел и создает из него тензор.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_tensor -v`

In [ ]:
!pytest . -k test_tensor -v

## Примеры использования

Теперь стоит проверить конструкторы, которые вы только что реализовали. Примеры их использования: создание нулевых векторов, векторов из единиц и векторов из списка координат. Все это конечно же создается в виде тензоров размерности 1. Запустите ячейку ниже, чтобы посмотреть, что получилось:

In [ ]:
import twinkle as tw

a = tw.zeros((2,))
b = tw.ones((5,))
c = tw.tensor([4, 2, 1])

print(a)
print(b)
print(c)
print()
print(tw.zeros((4,)))
print(tw.ones((3,)))
print(tw.tensor([7, 8, 9]))

# Математические операции (4.5 балла)

Теперь, когда реализован все вспомогательный функционал, можно перейти к самому главному -- реализовать математические операции над нашими тензорами-векторами. Все математические операции бинарные (сложение, умножение, вычитание и сравнение). Даже унарная операция взятия противоположного вектора может быть представлена как бинарная операция с умножением на `-1`.

Следует помнить, что все эти операции могут быть представлены разными типами данных. Один из операндов всегда наш объект класса `Tensor`, который мы пишем. Второй операнд может быть представлен объектами тремя различных типов:

1. Обычное питоновское число, `int` или `float`, в общем случае реализующее интерфейс `numbers.Number`;
2. Тензор нулевой размерности, фактически то же число, только в обертке тензора;
3. Тензор размерности такой же, как и у первого операнда. В этом случае операция проводится *поэлементно*.

*Подсказка: все эти операции очень похожи между собой, и можно реализовать единственный метод `_binary_op`, принимающий функцию, которую нужно поэлементно применять к элементам тензора, и второй операнд бинарной операции. Тогда останется только вызывать ее с правильными аргументами для каждой бинарной операции.*

Для удобства начнем с бинарной операции сложения и напишем ее по частям, после чего расширим наши знания на остальные бинарные операции, благо это будет просто.

## Сложение с числом, реализующим `numbers.Number` (0.5 балла)

Для начала реализуйте сложнение тензора с обычным питоновским числом. Сделать это можно в методе `add` внутри файла `twinkle/tensor.py`. Сложение -- простая бинарная операция, которая принимает тензор и питоновское число и возвращает тензор, координаты которого есть сумма соответствующих координат исходного вектора и заданного числа. Второй аргумент методе -- это `other`, где ожидается питоновское число. Обратите внимание, что внутри метода уже есть проверки на это, вам нужно дописать код в ветку под `isinstance(other, numbers.Number)`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_add_number -v`

In [ ]:
!pytest . -k test_add_number -v

## Сложение с тензором нулевой размерности (0.5 балла)

Теперь нужно реализовать сложение с тензором нулевой размерности. Это тоже простая бинарная операция, которая принимает тензор и тензор нулевой размерности и возвращает тензор, координаты которого есть сумма соответствующих координат исходного вектора и числа, спрятанного во втором тензоре. Второй аргумент метода -- это `other`, где ожидается тензор нулевой размерности. Обратите внимание, что внутри метода уже есть проверки на это, вам нужно дописать код в ветку под `isinstance(other, Tensor)`.

**Важно**: *здесь нужно дописать два случая: тензор нулевой размерности может лежать в `other`, а может и в `self`!*

## Перегрузка операторов

Основной функционал сложения мы уже реализовали, но пользоваться им неудобно, нужно звать метод `add`. Мы пишем буквы и скобки там, где было бы удобно пользоваться математической нотацией в виде знаков `+`, `*`, `-`, `>` или `<`.

В питоне для этого есть отдельный механизм, называемый *перегрузкой операторов*. Все операторы некоммутативны, то есть по умолчанию `a + b` не есть `b + a`. Для реализации левоассоциативных операторов нужно реализовать соответствующие методы в классе `Tensor`, а именно `__add__`, `__mul__` и `__sub__`. Для реализации правоассоциативных операторов нужно реализовать такие же методы, только с буквой `r` в начале, а именно `__radd__`, `__rmul__` и `__rsub__`. Каждый из этих методов принимает два аргумента бинарной операции, `self` и `other`.

Реализуйте магические методы `__add__` и `__radd__`, которые перегружают оператор бинарный оператор `+`. Их отличие в ассоциативности, `__add__` левоассоциативен, `__radd__` правоассоциативен. Внутри них надо просто позвать метод `add`, прокинув в него аргумент. Пример когда какой из них вызывается:

```python
foo = Tensor([0, 1, 2])
foo * 1  # вызывается __add__
1 * foo  # вызывается __radd__
```

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_add_scalar_tensor -v`

In [ ]:
!pytest . -k test_add_scalar_tensor -v

## Сложение двух тензоров одинаковой размерности (0.5 балла)

Ну и наконец сложение двух тензоров одинаковой размерности. Это тоже простая бинарная операция, которая принимает два тензора с одинаковым шейпом и возвращает тензор, координаты которого есть попарная сумма соответствующих координат исходных тензоров. Как и в предыдущем пункте вам нужно дописать код в ветку под `isinstance(other, Tensor)`, но уже под проверкой на одинаковые шейпы `self._assert_same_shape(other)`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_add_tensor -v`

In [ ]:
!pytest . -k test_add_tensor -v

Если все три предыдущих пункта прошли тесты, то сложение реализовано верно. Проверьте это, запустив тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_add -v`

In [ ]:
!pytest . -k test_add -v

## Метод `mul`, умножение (0.5 балла)

Аналогично сложению, реализуйте умножение тензора на число, нульмерный тензор и тензор такой же формы. Сделать это можно в методе `mul` внутри файла `twinkle/tensor.py`. Также реализуйте магические методы `__mul__` и `__rmul__` для перегрузки бинарного оператора `*`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_mul -v`

In [ ]:
!pytest . -k test_mul -v

## Метод `neg`, противоположный тензор (0.5 балла)

Порой для удобства необходимо брать противоположный вектор, в котором все координаты имеют противоположный знак. Это достаточно простая операция, которой можно сделать как напрямую, так и через умножение на `-1`. Сделать это можно в методе `sub` внутри файла `twinkle/tensor.py`. Также реализуйте магический метод `__neg__` для перегрузки унарного оператора `-`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_neg -v`

In [ ]:
!pytest . -k test_neg -v

## Метод `sub`, вычитание (0.5 балла)

Для удобства вычисления расстояний между векторами нам понадобится операция вычитания векторов, ведь это явно красивее, чем складывать векторы, где один имеет знак минус. Аналогично сложению, реализуйте вычитание из тензора числа, нульмерного тензора и тензора такой же формы. Сделать это можно в методе `sub` внутри файла `twinkle/tensor.py`. Также реализуйте магические методы `__sub__` и `__rsub__` для перегрузки бинарного оператора `-`.

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_sub -v`

In [ ]:
!pytest . -k test_sub -v

## Метод `dot`, (англ. dot product), скалярное произведение  (0.5 балла)

Скалярное произведение -- это бинарная операция, которая принимает на вход два вектора одинаковой длины и возвращает скаляр. Реализуйте скалярное произведение двух векторов в методе `dot` в файле `twinkle/tensor.py`.

*Подсказка: скалярное произведение -- это просто попарно перемножить координаты, а потом сложить что получилось. Это верно для ортонормированного базиса, а другое нас пока не интересует. Кого интересует -- см. матрица Грама.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_dot -v`

In [ ]:
!pytest . -k test_dot -v

## Методы `gt` и `lt`, сравнение (0.5 балла)

Иногда удобно поэлементно сравнивать тензор с числом или два тензора одинаковой формы. Результатом такой операции должна получаться маска сравнения, с единичками на тех позициях, где операция сравнения истинна, и с нулями там, где ложна. Реализуйте сравнение тензора с числом, нульмерным тензором и тензором такой же формы. Сделать это можно в методах `gt` и `lt` внутри файла `twinkle/tensor.py`. Также реализуйте магические методы `__gt__` и `__lt__` для перегрузки бинарных операторов `>` и `<`.

*Подсказка: если до этого вы все (кроме `dot`) делали через вызов `_binary_op`, то здесь можно сделать то же самое. Передавайте туда функцию, которая делает сравнение, а потом приводит результат к инту.*

Для проверки правильности своего решения запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -k test_gt -v`

In [ ]:
!pytest . -k test_gt -v

## Примеры использования

Создадим несколько векторов и посмотрим, как пользоваться полученными операциями. Обратите внимание, что тут есть различные операции с различными типами операндов.

In [ ]:
import twinkle as tw

print(tw.tensor(range(4)) + 1)
print(tw.tensor(range(4)) + tw.tensor(2))
print(tw.tensor(range(4)) + tw.tensor(range(4)), '\n')

print(tw.tensor(range(4)) * 1)
print(tw.tensor(range(4)) * tw.tensor(2))
print(tw.tensor(range(4)) * tw.tensor(range(4)), '\n')

print(tw.tensor(range(4)) - 1)
print(tw.tensor(range(4)) - tw.tensor(2))
print(tw.tensor(range(4)) - tw.tensor(range(4)), '\n')

print(tw.tensor(range(4)) > 1)
print(tw.tensor(range(4)) > tw.tensor(2))
print(tw.tensor(range(4)) > tw.tensor([-1, 0, 5, 5]))

Чтобы проверить все тесты сразу, запускайте тесты следующей командой (*и в ячейке ниже*):

`pytest . -v`

In [ ]:
!pytest . -v

## Бонус: аксиомы линейного пространства

Можно проверить аксиомы линейного пространства, запустив тест `test_axioms` или `test_stress_axioms`. Содержимое этих тестов можно изучить в ячейке ниже (она даже работает).

In [ ]:
import random
import twinkle as tw

test_shape = (random.randint(5, 10),)
alpha = random.gauss(0, 1)
beta = random.gauss(0, 1)

a, b, c = tw.randn(*test_shape), tw.randn(*test_shape), tw.randn(*test_shape)
zero = tw.zeros(test_shape)

assert a + b == b + a
assert (a + b) + c == a + (b + c)
assert a + zero == a
assert a + (-a) == zero

assert alpha * (beta * a) == (alpha * beta) * a
assert (alpha + beta) * a == alpha * a + beta * a
assert alpha * (a + b) == alpha * a + alpha * b
assert 1 * a == a

print('It works!')

# Резюме

В этой части мы написали основу векторной алгебры, а именно реализацию векторов и математических операций над ними.

Продолжение следует!

# Бонус: расстояние между векторами (1 балл)

Очень полезно при реализации метода k-ближайших соседей. Все, что нужно для вычисления расстояния между двумя векторами, у нас уже реализовано, осталось придумать, как совместить это в едином выражении. Создайте пару векторов и вычислите расстояние между ними, сделав это в ячейке ниже.

*Вопрос: как это сделать?*

*Подсказка: если у вас есть вычисления, тут явно что-то не то.*